In [63]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import os

In [74]:
# Hyper-parameters
num_epochs = 5
batch_size = 128
learning_rate = 0.001

input_size = 64
channel_size = 3

sequence_length = input_size * channel_size # input_size * input channel size = 64 * 3 => 192
hidden_size = 128
num_layers = 2

In [65]:
# data transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
    ]),
    'validation': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
    ]),
}

In [66]:
# Data loaders

data_directory = 'FMD2/Face Mask Dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_directory, x),
                                          data_transforms[x])
                  for x in ['train', 'validation', 'test']}


dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'validation', 'test']}

In [67]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation', 'test']}
print(dataset_sizes)

{'train': 10000, 'validation': 800, 'test': 992}


In [68]:
class_names = image_datasets['train'].classes
print(class_names)

['WithMask', 'WithoutMask']


In [69]:
num_classes = len(image_datasets['train'].classes)
print(num_classes)

2


In [70]:
from torch.autograd import Variable


class RNN(nn.Module):
    def __init__(self, in_dim, hid_dim, layer_dim, out_dim):
        super(RNN, self).__init__()

        self.hid_dim = hid_dim
        self.layer_dim = layer_dim

        self.rnn = nn.RNN(in_dim, hid_dim, layer_dim, batch_first=True, nonlinearity='relu')

        self.fC = nn.Linear(hid_dim, out_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hid_dim))

        out, hn = self.rnn(x, h0)
        out = self.fC(out[:, -1, :])
        return out

In [71]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [72]:
model

RNN(
  (rnn): RNN(64, 128, num_layers=2, batch_first=True)
  (fC): Linear(in_features=128, out_features=2, bias=True)
)

In [76]:
# Train Part

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloaders['train']):
        model.train()
        # Load images as tensors with gradient accumulation abilities
        images = images.view(-1, sequence_length, input_size).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

    model.eval()
    # Calculate Accuracy
    correct = 0
    total = 0

    # Iterate through validation dataset
    for images, labels in dataloaders['validation']:
        # Load images to a Torch tensors with gradient accumulation abilities
        images = images.view(-1, sequence_length, input_size)

        # Forward pass only to get logits/output
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Total number of labels
        total += labels.size(0)

        # Total correct predictions
        correct += (predicted == labels).sum()

    accuracy = 100 * correct / total

    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

Epoch: 0. Loss: 0.25696811079978943. Accuracy: 82.25
Epoch: 1. Loss: 0.35765373706817627. Accuracy: 82.125
Epoch: 2. Loss: 0.45775526762008667. Accuracy: 73.0
Epoch: 3. Loss: 0.28813236951828003. Accuracy: 82.5
Epoch: 4. Loss: 0.2359645962715149. Accuracy: 84.625


In [79]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in dataloaders['test']:
        images = images.reshape(-1, sequence_length, input_size)
        labels = labels
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the test images: {acc} %')

Accuracy of the network on the test images: 86.39112903225806 %
